**Table of contents**<a id='toc0_'></a>    
- [ロボットアーム](#toc1_)    
- [pybulletの起動](#toc2_)    
- [pybulletの初期設定](#toc3_)    
- [ロボットアームの生成](#toc4_)    
- [シミュレーションの実行](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[ロボットアーム](#toc0_)

本notebookでは2軸のロボットアームを生成し、制御する手順について解説します。

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physicsClient = pybullet.connect(pybullet.GUI) 

startThreads creating 1 threads.
starting thread 0


pybullet build time: Nov 28 2023 23:45:17


started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Microsoft Corporation
GL_RENDERER=D3D12 (AMD Radeon Graphics)
GL_VERSION=4.2 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.20
pthread_getconcurrency()=0
Version = 4.2 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Microsoft Corporation
Renderer = D3D12 (AMD Radeon Graphics)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


ven = Microsoft Corporation
ven = Microsoft Corporation


# <a id='toc3_'></a>[pybulletの初期設定](#toc0_)

In [9]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
timeStep = 1./240.
pybullet.setTimeStep(timeStep) # 1stepあたりに経過する時間の設定

#床の読み込み
planeId = pybullet.loadURDF("plane.urdf")

# GUIモードの際のカメラの位置などを設定
cameraDistance = 2.0
cameraYaw = 0.0 # deg
cameraPitch = -20 # deg
cameraTargetPosition = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

# <a id='toc4_'></a>[ロボットアームの生成](#toc0_)
今回は、2軸ロボットアーム`simple_2d_arm.urdf`を生成します。  
ロボットは下図のような構成になっています（センサーについては、`robot_arm_sensor.ipynb`にて解説しています。）  

![](../images/2d_robot_arm.png)

In [10]:
# ロボットの読み込み
armStartPos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
armStartOrientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
armId = pybullet.loadURDF("../urdf/simple2d_arm.urdf",armStartPos, armStartOrientation, useFixedBase=True) # ロボットが倒れないように、useFixedBase=Trueでルートのリンクを固定

# GUIモードの際のカメラの位置などを設定
cameraDistance = 1.5
cameraYaw = 180.0 # deg
cameraPitch = -10 # deg
cameraTargetPosition = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc5_'></a>[シミュレーションの実行](#toc0_)
`setJointMotorControl2` では、指定したロボットの指定したジョイントを動作させることができます。

最初に、速度指令で動作させるため、第三引数に`POSITION_CONTROL`を指定します。


In [4]:
import time
import math

# ロボットの各関節のインデックス
link1JointIdx = 0
link2JointIdx = 1

# 各関節の角度
joint1Angle = 90.0
joint2Angle = 90.0
joint1Rad = math.radians(joint1Angle)
joint2Rad = math.radians(joint2Angle)

for i in range(200):
    # 指定した角度に関節を動かす
    pybullet.setJointMotorControl2(armId, link1JointIdx, pybullet.POSITION_CONTROL, targetPosition=joint1Rad)
    pybullet.setJointMotorControl2(armId, link2JointIdx, pybullet.POSITION_CONTROL, targetPosition=joint2Rad)
    pybullet.stepSimulation()
    time.sleep(timeStep)

次に、速度指令で動作させるため、第三引数に`VELOCITY_CONTROL`を指定します。

In [11]:
joint1Velocity = 2.0
joint2Velocity = 2.0

for i in range(500):
    # 指定した速度で関節を動かす
    pybullet.setJointMotorControl2(armId, link1JointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=joint1Velocity)
    pybullet.setJointMotorControl2(armId, link2JointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=joint2Velocity)
    pybullet.stepSimulation()
    time.sleep(timeStep)

: 